In [1]:
import torch
from eg3d_dataset import EG3DDataset

In [2]:
dataset = EG3DDataset('data', triplanes=True)

In [3]:
dataset[0]['triplanes'].shape

torch.Size([96, 256, 256])

In [4]:
from diffusers import UNet2DConditionModel

2023-03-06 21:28:50.674545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 21:28:50.748337: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
diffuser = UNet2DConditionModel(
    sample_size=128,
    in_channels=96,
    out_channels=96,
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(64, 128, 256, 256),  # the number of output channes for each UNet block
    down_block_types = ('CrossAttnDownBlock2D', 'CrossAttnDownBlock2D', 'CrossAttnDownBlock2D', 'DownBlock2D'),
    up_block_types = ('UpBlock2D', 'CrossAttnUpBlock2D', 'CrossAttnUpBlock2D', 'CrossAttnUpBlock2D'),
)

In [ ]:
diffuser(dataset[0]['triplanes'].unsqueeze(0), 1, torch.tensor([10, 11, 12]))

In [6]:
diffuser = diffuser.to('cuda')

In [ ]:
out = diffuser(torch.zeros(1, 96, 128, 128).to('cuda'), 1, torch.zeros((1,1,1280)).to('cuda')).sample
out

In [10]:
lf = torch.nn.SmoothL1Loss()
lf(out, torch.ones(out.shape, device='cuda')).backward()


In [1]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

2023-03-06 22:04:18.739603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 22:04:18.812840: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
repo_id = "stabilityai/stable-diffusion-2-base"
# repo_id = "stabilityai/stable-diffusion-x4-upscaler"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
pipe.unet.block_out_channels

[320, 640, 1280, 1280]

In [ ]:
pipe.unet.sample_size